In [1]:
import polars as pl
import matplotlib.pyplot as plt
import datetime
import os

from utils.functions import tramit_file_reader, get_cars, dates_range, simplify_euro_emissions
from utils.dictionaries import types_parque_post, types_tramit_post
from utils.dictionaries import common_cols_tramites, common_cols_mapping, common_cols_parque_exacto

# Test of truncated files

## 1 month only

# Test of a whole file

## 1 month only (prior month)

### Result using the function

In [5]:
clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
necessary_cols = ['FECHA_PRIM_MATR','FECHA_MATR','PROPULSION']

park= pl.scan_csv(clean_park,separator='|', schema=types_parque_post).select(necessary_cols)
park = park.collect()
park = simplify_euro_emissions(park).drop('PROPULSION')
function_result = get_cars('01112023',park)
print(function_result)
print(function_result['count'].sum())

Fecha seleccionada: 2023-11-01, fecha de la foto: 2023-12-01
Estamos mirando una fecha anterior a la foto
Este es el parque en la foto exacta:
shape: (37_890_811, 3)
┌─────────────────┬────────────┬─────────────────┐
│ FECHA_PRIM_MATR ┆ FECHA_MATR ┆ Simplified_EURO │
│ ---             ┆ ---        ┆ ---             │
│ date            ┆ date       ┆ str             │
╞═════════════════╪════════════╪═════════════════╡
│ 2005-09-13      ┆ 2005-09-13 ┆ EURO_3          │
│ 2012-03-27      ┆ 2012-03-27 ┆ EURO_5          │
│ 2015-08-13      ┆ 2015-08-13 ┆ EURO_5          │
│ 2003-08-07      ┆ 2003-08-07 ┆ EURO_3          │
│ 2011-10-31      ┆ 2017-08-01 ┆ EURO_5          │
│ …               ┆ …          ┆ …               │
│ 2015-10-06      ┆ 2019-12-03 ┆ EURO_6          │
│ 2007-10-24      ┆ 2007-10-24 ┆ EURO_4          │
│ 1989-04-28      ┆ 1989-04-28 ┆ Previous        │
│ 2007-01-22      ┆ 2007-01-22 ┆ EURO_4          │
│ 2005-06-16      ┆ 2005-06-16 ┆ EURO_3          │
└─────────────────

### Result by manually checking

In [19]:
# The date is prior of the complete park so we 
# 1st need to get all the cars from complete_park and select the ones registered before the date
fecha = datetime.date(2023,11,1)
interest_park = park.filter(pl.col('FECHA_MATR') <= fecha)
interest_park = interest_park['Simplified_EURO'].value_counts().sort("Simplified_EURO")
interest_park

Simplified_EURO,count
str,u32
"""EURO_1""",897829
"""EURO_2""",2837695
"""EURO_3""",6777579
"""EURO_4""",7603642
"""EURO_5""",4594524
"""EURO_6""",11040728
"""EURO_CLEAN""",268454
"""Previous""",3618078


In [20]:
# Then we need to add up all the de-registrations of cars that where registered before the interest of date
# We know that for this case its only the ones in november 2023, we check if the function provides ir correctly too
file_of_interest = dates_range(datetime.date(2023,11,1), datetime.date(2023,12,1), 'bajas')
file_baja_path = os.path.join("..","Data", "DGT",'bajas',file_of_interest[0])
print(f'The file we are lloking is: {file_of_interest}')

common_cols_tramites = ['FEC_MATRICULA','FEC_PRIM_MATRICULACION','COD_PROPULSION_ITV']
common_cols_mapping = {"FEC_MATRICULA": "FECHA_MATR", "FEC_PRIM_MATRICULACION": "FECHA_PRIM_MATR", "COD_PROPULSION_ITV": "PROPULSION"}

bajas = tramit_file_reader(file_baja_path).select(common_cols_tramites).rename(common_cols_mapping)
bajas.filter(pl.col('FECHA_MATR') <= fecha)
bajas = bajas.with_columns([
    pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
    .otherwise(pl.col("FECHA_PRIM_MATR"))
    .alias("FECHA_PRIM_MATR")])

bajas = simplify_euro_emissions(bajas).drop('PROPULSION')
bajas = bajas['Simplified_EURO'].value_counts().sort("Simplified_EURO")
bajas

The file we are lloking is: ['export_mensual_bajas_202311.txt']


Simplified_EURO,count
str,u32
"""EURO_1""",5639
"""EURO_2""",18782
"""EURO_3""",39372
"""EURO_4""",28835
"""EURO_5""",13317
"""EURO_6""",45148
"""Previous""",7501


In [21]:
# So we have the cars from the park
print('Registered cars')
print(interest_park)
# And the cars from the de-registrations
print('De-registrations cars')
print(bajas)

Registered cars
shape: (8, 2)
┌─────────────────┬──────────┐
│ Simplified_EURO ┆ count    │
│ ---             ┆ ---      │
│ str             ┆ u32      │
╞═════════════════╪══════════╡
│ EURO_1          ┆ 897829   │
│ EURO_2          ┆ 2837695  │
│ EURO_3          ┆ 6777579  │
│ EURO_4          ┆ 7603642  │
│ EURO_5          ┆ 4594524  │
│ EURO_6          ┆ 11040728 │
│ EURO_CLEAN      ┆ 268454   │
│ Previous        ┆ 3618078  │
└─────────────────┴──────────┘
De-registrations cars
shape: (7, 2)
┌─────────────────┬───────┐
│ Simplified_EURO ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ EURO_1          ┆ 5639  │
│ EURO_2          ┆ 18782 │
│ EURO_3          ┆ 39372 │
│ EURO_4          ┆ 28835 │
│ EURO_5          ┆ 13317 │
│ EURO_6          ┆ 45148 │
│ Previous        ┆ 7501  │
└─────────────────┴───────┘


In [34]:
# If we add the up:
park_distribution = (pl.concat([interest_park, bajas]))
with pl.Config(tbl_rows=16):
    print(park_distribution)

shape: (15, 2)
┌─────────────────┬──────────┐
│ Simplified_EURO ┆ count    │
│ ---             ┆ ---      │
│ str             ┆ u32      │
╞═════════════════╪══════════╡
│ EURO_1          ┆ 897829   │
│ EURO_2          ┆ 2837695  │
│ EURO_3          ┆ 6777579  │
│ EURO_4          ┆ 7603642  │
│ EURO_5          ┆ 4594524  │
│ EURO_6          ┆ 11040728 │
│ EURO_CLEAN      ┆ 268454   │
│ Previous        ┆ 3618078  │
│ EURO_1          ┆ 5639     │
│ EURO_2          ┆ 18782    │
│ EURO_3          ┆ 39372    │
│ EURO_4          ┆ 28835    │
│ EURO_5          ┆ 13317    │
│ EURO_6          ┆ 45148    │
│ Previous        ┆ 7501     │
└─────────────────┴──────────┘


In [44]:
print('Group by Simplified_EURO')
park_distribution = park_distribution.group_by('Simplified_EURO').agg(pl.sum("count")).sort('Simplified_EURO')
print(park_distribution)

Group by Simplified_EURO
shape: (8, 2)
┌─────────────────┬──────────┐
│ Simplified_EURO ┆ count    │
│ ---             ┆ ---      │
│ str             ┆ u32      │
╞═════════════════╪══════════╡
│ EURO_1          ┆ 897987   │
│ EURO_2          ┆ 2838048  │
│ EURO_3          ┆ 6779301  │
│ EURO_4          ┆ 7609511  │
│ EURO_5          ┆ 4602734  │
│ EURO_6          ┆ 11375985 │
│ EURO_CLEAN      ┆ 285016   │
│ Previous        ┆ 3619265  │
└─────────────────┴──────────┘


It is ok

## 1 month only (posterior month)

### Result using the function

In [38]:
clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
necessary_cols = ['FECHA_PRIM_MATR','FECHA_MATR','PROPULSION']

park= pl.scan_csv(clean_park,separator='|', schema=types_parque_post).select(necessary_cols)
park = park.collect()
park = simplify_euro_emissions(park).drop('PROPULSION')
function_result = get_cars('01012024',park)
print(function_result)
print(function_result['count'].sum())

Fecha seleccionada: 2024-01-01, fecha de la foto: 2023-12-01
Estamos mirando una fecha posterior a la foto
Este es el parque exacto:
shape: (37_890_811, 3)
┌─────────────────┬────────────┬─────────────────┐
│ FECHA_PRIM_MATR ┆ FECHA_MATR ┆ Simplified_EURO │
│ ---             ┆ ---        ┆ ---             │
│ date            ┆ date       ┆ str             │
╞═════════════════╪════════════╪═════════════════╡
│ 2005-09-13      ┆ 2005-09-13 ┆ EURO_3          │
│ 2012-03-27      ┆ 2012-03-27 ┆ EURO_5          │
│ 2015-08-13      ┆ 2015-08-13 ┆ EURO_5          │
│ 2003-08-07      ┆ 2003-08-07 ┆ EURO_3          │
│ 2011-10-31      ┆ 2017-08-01 ┆ EURO_5          │
│ …               ┆ …          ┆ …               │
│ 2015-10-06      ┆ 2019-12-03 ┆ EURO_6          │
│ 2007-10-24      ┆ 2007-10-24 ┆ EURO_4          │
│ 1989-04-28      ┆ 1989-04-28 ┆ Previous        │
│ 2007-01-22      ┆ 2007-01-22 ┆ EURO_4          │
│ 2005-06-16      ┆ 2005-06-16 ┆ EURO_3          │
└─────────────────┴─────────

### Result by manually checking

In [59]:
# The date is posterior of the complete park so we 
# 1st need to get all the cars from complete_park
fecha = datetime.date(2024,1,1)
interest_park = park.filter(pl.col('FECHA_MATR') <= fecha)
interest_park = interest_park['Simplified_EURO'].value_counts().sort("Simplified_EURO")
interest_park

Simplified_EURO,count
str,u32
"""EURO_1""",897930
"""EURO_2""",2837921
"""EURO_3""",6778700
"""EURO_4""",7607502
"""EURO_5""",4599992
"""EURO_6""",11264865
"""EURO_CLEAN""",285016
"""Previous""",3618885


In [60]:
# Then we need to add up all the registrations of cars that untill the date
# We know that for this case its only the ones in january 2024, we check if the function provides ir correctly too
file_of_interest = dates_range(datetime.date(2023,12,1), fecha, 'mat')
file_mat_path = os.path.join("..","Data", "DGT",'matr',file_of_interest[0])
print(f'The file we are lloking is: {file_of_interest}')

common_cols_tramites = ['FEC_MATRICULA','FEC_PRIM_MATRICULACION','COD_PROPULSION_ITV']
common_cols_mapping = {"FEC_MATRICULA": "FECHA_MATR", "FEC_PRIM_MATRICULACION": "FECHA_PRIM_MATR", "COD_PROPULSION_ITV": "PROPULSION"}

mat = tramit_file_reader(file_mat_path).select(common_cols_tramites).rename(common_cols_mapping)

mat = mat.with_columns([
    pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
    .otherwise(pl.col("FECHA_PRIM_MATR"))
    .alias("FECHA_PRIM_MATR")])

mat = simplify_euro_emissions(mat).drop('PROPULSION')
mat = mat['Simplified_EURO'].value_counts().sort("Simplified_EURO")
mat

The file we are lloking is: ['export_mensual_mat_202401.txt']


Simplified_EURO,count
str,u32
"""EURO_1""",57
"""EURO_2""",127
"""EURO_3""",601
"""EURO_4""",2009
"""EURO_5""",2742
"""EURO_6""",111120
"""Previous""",380


In [61]:
# If we add the up:
park_distribution = (pl.concat([interest_park, mat]))
with pl.Config(tbl_rows=16):
    print(park_distribution)

shape: (15, 2)
┌─────────────────┬──────────┐
│ Simplified_EURO ┆ count    │
│ ---             ┆ ---      │
│ str             ┆ u32      │
╞═════════════════╪══════════╡
│ EURO_1          ┆ 897930   │
│ EURO_2          ┆ 2837921  │
│ EURO_3          ┆ 6778700  │
│ EURO_4          ┆ 7607502  │
│ EURO_5          ┆ 4599992  │
│ EURO_6          ┆ 11264865 │
│ EURO_CLEAN      ┆ 285016   │
│ Previous        ┆ 3618885  │
│ EURO_1          ┆ 57       │
│ EURO_2          ┆ 127      │
│ EURO_3          ┆ 601      │
│ EURO_4          ┆ 2009     │
│ EURO_5          ┆ 2742     │
│ EURO_6          ┆ 111120   │
│ Previous        ┆ 380      │
└─────────────────┴──────────┘


In [62]:
print('Group by Simplified_EURO')
park_distribution = park_distribution.group_by('Simplified_EURO').agg(pl.sum("count")).sort('Simplified_EURO')
park_distribution

Group by Simplified_EURO


Simplified_EURO,count
str,u32
"""EURO_1""",897987
"""EURO_2""",2838048
"""EURO_3""",6779301
"""EURO_4""",7609511
"""EURO_5""",4602734
"""EURO_6""",11375985
"""EURO_CLEAN""",285016
"""Previous""",3619265


In [63]:
# Now we have to substract the de-registrations of cars.
# We need to account the ones with registration dates older than the exact park for crrecting that assumption
# And also the ones with older registration dates to correct considerieng all the registrations
file_of_interest = dates_range(datetime.date(2023,12,1), fecha, 'bajas')
file_bajas_path = os.path.join("..","Data", "DGT",'bajas',file_of_interest[0])
print(f'The file we are lloking is: {file_of_interest}')

common_cols_tramites = ['FEC_MATRICULA','FEC_PRIM_MATRICULACION','COD_PROPULSION_ITV']
common_cols_mapping = {"FEC_MATRICULA": "FECHA_MATR", "FEC_PRIM_MATRICULACION": "FECHA_PRIM_MATR", "COD_PROPULSION_ITV": "PROPULSION"}

bajas = tramit_file_reader(file_bajas_path).select(common_cols_tramites).rename(common_cols_mapping)

bajas = bajas.with_columns([
    pl.when(pl.col("FECHA_PRIM_MATR").is_null()).then(pl.col("FECHA_MATR"))
    .otherwise(pl.col("FECHA_PRIM_MATR"))
    .alias("FECHA_PRIM_MATR")])

bajas = simplify_euro_emissions(bajas).drop('PROPULSION')
bajas = bajas['Simplified_EURO'].value_counts().sort("Simplified_EURO")
bajas = bajas.with_columns((pl.col("count") * -1).alias("count"))
bajas

The file we are lloking is: ['export_mensual_bajas_202401.txt']


Simplified_EURO,count
str,i64
"""EURO_1""",-4553
"""EURO_2""",-15158
"""EURO_3""",-32122
"""EURO_4""",-23464
"""EURO_5""",-10630
"""EURO_6""",-35456
"""Previous""",-6132


In [64]:
# If we add the up:
park_distribution = park_distribution.with_columns(
    pl.col("count").cast(pl.Int64))

park_distribution = (pl.concat([park_distribution, bajas]))
with pl.Config(tbl_rows=16):
    print(park_distribution)

shape: (15, 2)
┌─────────────────┬──────────┐
│ Simplified_EURO ┆ count    │
│ ---             ┆ ---      │
│ str             ┆ i64      │
╞═════════════════╪══════════╡
│ EURO_1          ┆ 897987   │
│ EURO_2          ┆ 2838048  │
│ EURO_3          ┆ 6779301  │
│ EURO_4          ┆ 7609511  │
│ EURO_5          ┆ 4602734  │
│ EURO_6          ┆ 11375985 │
│ EURO_CLEAN      ┆ 285016   │
│ Previous        ┆ 3619265  │
│ EURO_1          ┆ -4553    │
│ EURO_2          ┆ -15158   │
│ EURO_3          ┆ -32122   │
│ EURO_4          ┆ -23464   │
│ EURO_5          ┆ -10630   │
│ EURO_6          ┆ -35456   │
│ Previous        ┆ -6132    │
└─────────────────┴──────────┘


In [65]:
print('Group by Simplified_EURO')
park_distribution = park_distribution.group_by('Simplified_EURO').agg(pl.sum("count")).sort('Simplified_EURO')
(park_distribution)

Group by Simplified_EURO


Simplified_EURO,count
str,i64
"""EURO_1""",893434
"""EURO_2""",2822890
"""EURO_3""",6747179
"""EURO_4""",7586047
"""EURO_5""",4592104
"""EURO_6""",11340529
"""EURO_CLEAN""",285016
"""Previous""",3613133


In [66]:
clean_park = os.path.join("..","Data", "DGT",'Parque_exacto','clean_park.csv')
necessary_cols = ['FECHA_PRIM_MATR','FECHA_MATR','PROPULSION']

park= pl.scan_csv(clean_park,separator='|', schema=types_parque_post).select(necessary_cols)
park = park.collect()
park = simplify_euro_emissions(park).drop('PROPULSION')
function_result = get_cars('01012024',park)
print(function_result)
print(function_result['count'].sum())

Fecha seleccionada: 2024-01-01, fecha de la foto: 2023-12-01
Estamos mirando una fecha posterior a la foto
Este es el parque exacto:
shape: (37_890_811, 3)
┌─────────────────┬────────────┬─────────────────┐
│ FECHA_PRIM_MATR ┆ FECHA_MATR ┆ Simplified_EURO │
│ ---             ┆ ---        ┆ ---             │
│ date            ┆ date       ┆ str             │
╞═════════════════╪════════════╪═════════════════╡
│ 2005-09-13      ┆ 2005-09-13 ┆ EURO_3          │
│ 2012-03-27      ┆ 2012-03-27 ┆ EURO_5          │
│ 2015-08-13      ┆ 2015-08-13 ┆ EURO_5          │
│ 2003-08-07      ┆ 2003-08-07 ┆ EURO_3          │
│ 2011-10-31      ┆ 2017-08-01 ┆ EURO_5          │
│ …               ┆ …          ┆ …               │
│ 2015-10-06      ┆ 2019-12-03 ┆ EURO_6          │
│ 2007-10-24      ┆ 2007-10-24 ┆ EURO_4          │
│ 1989-04-28      ┆ 1989-04-28 ┆ Previous        │
│ 2007-01-22      ┆ 2007-01-22 ┆ EURO_4          │
│ 2005-06-16      ┆ 2005-06-16 ┆ EURO_3          │
└─────────────────┴─────────

Its OK